Базовая нейронка для обработки абстрактного набора числовых данных

https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0

In [5]:
import torch
import torch.nn as nn

In [6]:
!nvidia-smi

Fri Dec 30 19:24:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0B:00.0 Off |                  N/A |
|  0%   59C    P0   104W / 350W |      0MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Set device type
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor

In [7]:
print(X.size())
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


In [8]:
# scale units
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100  # max test score is 100

In [17]:
# Tensor not on GPU
print(y, y.device)

# Move tensor to GPU (if available)
X_on_gpu = X.to(device)
xPredicted_on_gpu = xPredicted.to(device)
y_on_gpu = y.to(device)

tensor([[0.9200],
        [1.0000],
        [0.8900]]) cpu


In [19]:
class Neural_Network(nn.Module):
    def __init__(self, device):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize).to(device) # 2 X 3 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize).to(device) # 3 X 1 tensor

        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self, X_test):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(X_test))
        print ("Output: \n" + str(self.forward(X_test)))

In [21]:

NN = Neural_Network(device)
# for i in range(1000):  # trains the NN 1,000 times
#     print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
#     NN.train(X, y)
for i in range(1000):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y_on_gpu - NN(X_on_gpu))**2).detach().item()))  # mean sum squared loss
    NN.train(X_on_gpu, y_on_gpu)
NN.saveWeights(NN)
# NN.predict(xPredicted)
NN.predict(xPredicted_on_gpu)

#0 Loss: 0.3143742084503174
#1 Loss: 0.22764277458190918
#2 Loss: 0.16210246086120605
#3 Loss: 0.11674009263515472
#4 Loss: 0.08628164231777191
#5 Loss: 0.06572853773832321
#6 Loss: 0.05156044661998749
#7 Loss: 0.04152767360210419
#8 Loss: 0.03422677516937256
#9 Loss: 0.028776641935110092
#10 Loss: 0.024613453075289726
#11 Loss: 0.02136765420436859
#12 Loss: 0.018791088834404945
#13 Loss: 0.016712982207536697
#14 Loss: 0.01501321792602539
#15 Loss: 0.013605514541268349
#16 Loss: 0.01242670975625515
#17 Loss: 0.011429771780967712
#18 Loss: 0.010579163208603859
#19 Loss: 0.009847555309534073
#20 Loss: 0.009213757701218128
#21 Loss: 0.008661062456667423
#22 Loss: 0.008176223374903202
#23 Loss: 0.007748562376946211
#24 Loss: 0.007369435857981443
#25 Loss: 0.007031796034425497
#26 Loss: 0.006729805842041969
#27 Loss: 0.006458630785346031
#28 Loss: 0.006214238237589598
#29 Loss: 0.005993237718939781
#30 Loss: 0.0057927402667701244
#31 Loss: 0.005610307212918997
#32 Loss: 0.005443856120109558